# AI safety via debate
OAI article: https://openai.com/index/debate/

Paper: https://arxiv.org/pdf/1805.00899

Implementation by Alex Chen

Jul 2024

# Setup

In [4]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from torchvision.transforms import ToTensor

# Training the Judge
The judge is a classifier trained to "predict MNIST digits from 6 non-black digits, sampled at random for each presentation when pretraining."

As per the paper, their original sparse classifier achieved 59.4% accuracy. They use a convolutional neural net.

In [2]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.0005
momentum = 0.5
log_interval = 10

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torchvision.transforms import ToTensor

class RandomPixelSelection(object):
    def __init__(self, num_pixels=6):
        self.num_pixels = num_pixels

    def __call__(self, img):
        img_np = np.array(img)
        flattened = img_np.flatten()
        non_black_indices = np.where(flattened > 0)[0]

        if len(non_black_indices) < self.num_pixels:
            selected_indices = non_black_indices
        else:
            selected_indices = np.random.choice(non_black_indices, self.num_pixels, replace=False)

        new_img = np.zeros_like(flattened)
        new_img[selected_indices] = flattened[selected_indices]
        new_img = new_img.reshape(img_np.shape)

        new_img = torch.tensor(new_img, dtype=torch.float32)
        new_img = (new_img - 0.1307) / 0.3081

        return new_img.unsqueeze(0)

transform = torchvision.transforms.Compose([
    RandomPixelSelection(num_pixels=6)
])

batch_size_train = 64
batch_size_test = 1000

train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('/files/', train=True, download=True, transform=transform),
    batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('/files/', train=False, download=True, transform=transform),
    batch_size=batch_size_test, shuffle=True)

class Judge(nn.Module):
    def __init__(self):
        super(Judge, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 12, kernel_size=5)
        self.fc1 = nn.Linear(12 * 20 * 20, 60)
        self.fc2 = nn.Linear(60, 30)
        self.fc3 = nn.Linear(30, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 12 * 20 * 20)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Judge().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.7)

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)\n')

n_epochs = 20
for epoch in range(1, n_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

# The Agent

use monte carlo tree search

tree layers alternate between the truthful agent and the lying agent

each node can track the:
- parent
- children
- full game state up to that node, which also reveals which agent they are and what turn it is
- value; number of wins


the root of the tree is the empty state

first move is truthful agent

In [1]:
class Node():
    def __init__(self, parent=None, children=[], gameState=[], value=0, turn=0):
        self.parent = parent
        self.children = children
        self.gameState = gameState
        self.value = value

In [2]:
class MCTS():
    def __init__(self):
        self.root = Node(parent=None, value=2)
    
    def chooseNextNode(self, curNode):
        temp = []
        if len(curNode.children) == 0:
            return curNode
        for c in curNode.children:
            temp += c.value
        return random.choices(population=curNode.children, weights=temp, k=1)

    def select(self):
        curNode = self.root
        while curNode is not None:
            nextNode = self.chooseNextNode(curNode)
            if nextNode != curNode:
                curNode = nextNode
            else:
                break

    def backprop(self, node, value):
        node.value += value
        while node.parent is not None:
            node.parent.value += value
            node = node.parent

In [3]:
a = MCTS()
print(a.getRoot())

2
